<a href="https://colab.research.google.com/github/kimberley-faria/implicit-hate-detection/blob/main/implicit_hate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

setup a git repo in your google drive - https://medium.com/analytics-vidhya/how-to-use-google-colab-with-github-via-google-drive-68efb23a42d

(I did this in a separate notebook, so I can keep this one free of git commit/push etc commands)

In [1]:
# if local
!conda activate cs685

In [2]:
import os
import sys
os.path.dirname(sys.executable)

'C:\\Users\\faria\\anaconda3\\envs\\cs685'

In [3]:
import sys

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = '/content/drive/MyDrive/Github/685/implicit-hate-detection'
else:
    base_path = r'C:\Users\faria\PycharmProjects\685\project\implicit-hate-detection'

switch to the git repo in your drive

In [4]:
%cd $base_path
!git checkout 2-bert-baselines-bin-classification

C:\Users\faria\PycharmProjects\685\project\implicit-hate-detection
M	README.md
Your branch is up to date with 'origin/2-bert-baselines-bin-classification'.


Already on '2-bert-baselines-bin-classification'


In [5]:
!git status

On branch 2-bert-baselines-bin-classification
Your branch is up to date with 'origin/2-bert-baselines-bin-classification'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   README.md

no changes added to commit (use "git add" and/or "git commit -a")


install requirements from repo 

In [6]:
# for colab, use venv if in local
# !pip install -r requirements.txt

load the helper and dataloader files

In [7]:
%load implicit_hate_dataloader/dataloader.py    
%load implicit_hate_dataloader/helpers.py

In [8]:
import torch
assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: GeForce GTX 1050 Ti with Max-Q Design, n_gpu: 1


In [9]:
import os

data_path = '../data/implicit-hate-corpus-nov-2021/implicit-hate-corpus'
dataset_filename = {
    # post (str)
    # class (str): high-level label in {`explicit_hate`,`implicit_hate`,`not_hate`}
    "stage-1": os.path.join(data_path, "implicit_hate_v1_stg1_posts.tsv"),

    # post (str)
    # class (str): fine-grained implicit hate label in
    # {`white_grievance`, `incitement`, `inferiority`, `irony`, `stereotypical`, `threatening`, `other`}
    # extra_implicit_class: (str) A secondary fine-grained implicit hate label in
    # {`white_grievance`, `incitement`, `inferiority`, `irony`, `stereotypical`, `threatening`, `other`, **None**}
    "stage-2": os.path.join(data_path, "implicit_hate_v1_stg2_posts.tsv"),

    # post (str)
    # target: (str) Free-text annotation for the group being targeted (e.g. `Black people`, `Immigrants`, etc.)
    # implied_statement: (str) Free-text annotation for the implicit or hidden underlying meaning of the post made
    # explicit (e.g. `people in minority groups are all in gangs`)
    "stage-3": os.path.join(data_path, "implicit_hate_v1_stg3_posts.tsv")
}

In [10]:
from implicit_hate_dataloader.dataloader import Stage1Dataset, Stage2Dataset
from torch.utils.data import DataLoader

In [13]:
# Stage 1 Annotations: see config.py for dataset specs
implicit_hate_dataset = Stage1Dataset(dataset_filename["stage-1"], drop_explicit_hate=True)

dataset_size = len(implicit_hate_dataset)
dataset_splits = [int(0.6*dataset_size), int(0.2*dataset_size), dataset_size - int(0.6*dataset_size) - int(0.2*dataset_size)]

# # Smaller dataset for testing model
# dataset_splits = [int(0.006*dataset_size), int(0.002*dataset_size), int(0.002*dataset_size)]
print("Dataset size", dataset_size)
print("Dataset splits", dataset_splits)

train_set, val_set, test_set = torch.utils.data.random_split(implicit_hate_dataset, dataset_splits)

train_dataloader =  DataLoader(train_set, batch_size=4, shuffle=False)
validation_dataloader = DataLoader(val_set, batch_size=4, shuffle=False)

# assert len(train_set) == int(0.6*dataset_size)
# assert len(val_set) == int(0.2*dataset_size)

Dataset size 20391
Dataset splits [12234, 4078, 4079]


In [14]:
# Confirming type is tensor
_, _, input_ids, attention_masks, labels = next(iter(train_dataloader))
print(type(input_ids), type(attention_masks), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>


In [15]:
import numpy as np
# function to get validation accuracy
def get_validation_performance(model, val_set, config):
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0

    validation_dataloader = DataLoader(val_set, batch_size=config["batch_size"], shuffle=True)

    total_correct = 0

    for batch in validation_dataloader:

      input_id_tensors = batch[2]
      input_mask_tensors = batch[3]
      label_tensors = batch[4]
      
      # Move tensors to the GPU
      b_input_ids = input_id_tensors.to(device)
      b_input_mask = input_mask_tensors.to(device)
      b_labels = label_tensors.to(device)
        
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask,
                        labels=b_labels)
        loss = outputs.loss
        logits = outputs.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the number of correctly labeled examples in batch
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        num_correct = np.sum(pred_flat == labels_flat)
        total_correct += num_correct
        
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_correct / len(val_set)
    return avg_val_accuracy

In [16]:
!wandb login 

wandb: Currently logged in as: umass-iesl-is (use `wandb login --relogin` to force relogin)


In [17]:
import wandb

# sweep_config = {
#     "method" : "grid",
#     "parameters" : {
#         "batch_size": {
#             "values" : [8, 16, 32, 64]
#         },
#         "learning_rate" : {
#             "values": [2e-5, 3e-5, 5e-5]
#         },
#         "epochs" : {
#             "values" : [1, 2, 3, 4, 5]
#         },
#     }
# }

# hyperparameters used in original paper
sweep_config = {
    "name": "original-ppr-hparams-sweep",
    "method" : "grid",
    "parameters" : {
        "batch_size": {
            "values" : [8]
        },
        "learning_rate" : {
            "values": [2e-5, 3e-5, 5e-5]
        },
        "epochs" : {
            "values" : [1, 2, 3, 4]
        },
        "epsilon": {
            "values" : [1e-8]
        },
    }
}

sweep_id = wandb.sweep(sweep_config, entity="umass-iesl-is", project="cs685-project")

Create sweep with ID: honyh34l
Sweep URL: https://wandb.ai/umass-iesl-is/cs685-project/sweeps/honyh34l


In [18]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

def train():
    with wandb.init() as run:
        config = wandb.config
        
        model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
        num_labels = 3, # The number of output labels.   
        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.
        )

        # Tell pytorch to run this model on the GPU.
        model.cuda()


        optimizer = AdamW(model.parameters(),
                        lr = config["learning_rate"], # args.learning_rate - default is 5e-5
                        eps = config["epsilon"] # args.adam_epsilon  - default is 1e-8
                        )
        wandb.watch(model, log="all")


        for epoch_i in range(0, config["epochs"]):
            # Perform one full pass over the training set.

            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, config["epochs"]))
            print('Training...')

            # Reset the total loss for this epoch.
            total_train_loss = 0

            # Put the model into training mode.
            model.train()

            # For each batch of training data...
            train_dataloader =  DataLoader(train_set, batch_size=config["batch_size"], shuffle=False)

            for batch_idx, batch in enumerate(train_dataloader):

                input_id_tensors = batch[2]
                input_mask_tensors = batch[3]
                label_tensors = batch[4]

                # Move tensors to the GPU
                b_input_ids = input_id_tensors.to(device)
                b_input_mask = input_mask_tensors.to(device)
                b_labels = label_tensors.to(device)

                # Clear the previously calculated gradient
                model.zero_grad()        

                # Perform a forward pass (evaluate the model on this training batch).
                outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask, 
                                labels=b_labels)
                loss = outputs.loss
                logits = outputs.logits

                total_train_loss += loss.item()

                # Perform a backward pass to calculate the gradients.
                loss.backward()

                # Update parameters and take a step using the computed gradient.
                optimizer.step()
                wandb.log({
                    "batch_loss": loss.item(),
                    "batch": batch_idx
                })
                
            # ========================================
            #               Validation
            # ========================================
            # After the completion of each training epoch, measure our performance on
            # our validation set. Implement this function in the cell above.
            print(f"Total loss: {total_train_loss}")
            val_acc = get_validation_performance(model, val_set, config)
            print(f"Validation accuracy: {val_acc}")
            wandb.log({
                "loss": total_train_loss, 
                "epoch": config["epochs"], 
                "val_acc": val_acc
                })
            
        print("")
        print("Training complete!")

        test_acc = get_validation_performance(model, test_set, config)
        print(f"Test accuracy: {test_acc}")
        wandb.log({"test_acc": test_acc})

        # torch.save(model.state_dict(), "model.h5")
        # wandb.save('model.h5')

In [19]:
count = 12 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count, entity="umass-iesl-is", project="cs685-project")

wandb: Agent Starting Run: toe7hdaq with config:
wandb: 	batch_size: 8
wandb: 	epochs: 1
wandb: 	epsilon: 1e-08
wandb: 	learning_rate: 2e-05
wandb: Currently logged in as: umass-iesl-is (use `wandb login --relogin` to force relogin)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


======== Epoch 1 / 1 ========
Training...
Total loss: 792.6600777581334
Validation accuracy: 0.8001471309465424

Training complete!
Test accuracy: 0.7830350576121599



batch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch_loss,██▇▆▅▆▅▆▅▂▆▆▄▂▃▃▃▅▁▅▃▇▅▇▇▅█▁▃▂▃▄▅▅▂▆▃▂▆▃
epoch,▁
loss,▁
test_acc,▁
val_acc,▁
batch,1529
batch_loss,0.06216
epoch,1
loss,792.66008
test_acc,0.78304


wandb: Agent Starting Run: 27ef9kjv with config:
wandb: 	batch_size: 8
wandb: 	epochs: 1
wandb: 	epsilon: 1e-08
wandb: 	learning_rate: 3e-05


...

batch,▁▂▂▃▄▅▅▆▇█▁▂▂▃▄▅▅▆▇█▁▂▃▃▄▅▅▆▇█▁▂▃▃▄▅▅▆▇█
batch_loss,█▇▇▂▄▄▅▄▆██▅▃▄▃▅▃▃▆▆▂▂▂▅▆▄▁▅▇▄▃▁▁▂▃▂▁▁▂▁
epoch,▁▁▁▁
loss,█▅▂▁
test_acc,▁
val_acc,██▃▁
batch,1529
batch_loss,0.00091
epoch,4
loss,174.31114
test_acc,0.76416


wandb: Agent Starting Run: sy7yh1gs with config:
wandb: 	batch_size: 8
wandb: 	epochs: 4
wandb: 	epsilon: 1e-08
wandb: 	learning_rate: 5e-05


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


======== Epoch 1 / 4 ========
Training...
Total loss: 819.6193683370948
Validation accuracy: 0.7883766552231486

======== Epoch 2 / 4 ========
Training...
Total loss: 588.247581332922
Validation accuracy: 0.7736635605689063

======== Epoch 3 / 4 ========
Training...
Total loss: 384.747281906195
Validation accuracy: 0.7766061794997547

======== Epoch 4 / 4 ========
Training...
Total loss: 256.90393599972595
Validation accuracy: 0.7663070132417852

Training complete!
Test accuracy: 0.7457710223093895



batch,▁▂▂▃▄▅▅▆▇█▁▂▂▃▄▅▅▆▇█▁▂▃▃▄▅▅▆▇█▁▂▃▃▄▅▅▆▇█
batch_loss,▇▆▆▂▄▃▄▄▅██▆▄▅▂▃▂▃▅▄▂▁▂▃▂▃▂▂▃▃▄▁▇▃▁▅▁▁▄▁
epoch,▁▁▁▁
loss,█▅▃▁
test_acc,▁
val_acc,█▃▄▁
batch,1529
batch_loss,0.02113
epoch,4
loss,256.90394
test_acc,0.74577
